## 10.1 제네릭타입 이해하기
* 제네릭타이븐 인터페이스나 클래스, 함수, 타입별칭등에 사용할 수 있는 기능
* 해당 심벌타입을 사전에 지정하지 않고 다양한 타입에 대응하도록 할 때 사용

### 제네릭타입 예
```ts
// 제네릭 인터페이스 구문
interface IValuable<T> {
  value:
  T
}

// 제네릭함수
function  identity<T>(arg: T): T { return arg }

// 제네릭타입별칭
type IValuable<T> {
  value: T
}

// 제네릭클래스
class Valuable<T> {
  constructor(public value: T) {}
}
```

#### 제네릭 사용하기
* 제네릭 인터페이스

##### src/1001/IValuable.ts

```ts
export interface IValuable<T> {
  value:
  T
}
```
##### src/1001/Valuable.ts

```ts
import { IValuable } from './IValuable'

export class Valuable<T> implements IValuable<T> {
  constructor(public value: T) {}
}

export { IValuable }
```

##### src/1001/printValue.ts

```ts
import { IValuable, Valuable } from './Valuable'

export const printValue = <T>(o: IValuable<T>): void => console.log(o.value)
export { IValuable, Valuable }
```

##### src/1001/printValue-test.ts

```ts
import { printValue, Valuable } from './printValue'

printValue(new Valuable<number>(1)) // 1
printValue(new Valuable<boolean>(true)) // true
printValue(new Valuable<string>('hello')) // hello
printValue(
  new Valuable<number[]>([1, 2, 3])
) // [1, 2, 3]

```

##### src/1001/printValue-test2.ts
* 제네릭타입을 생략해도 스스로 추론해서 구체적인 제네릭타입을 찾아낸다.

```ts
import { printValue, Valuable } from './printValue'

printValue(new Valuable(1)) // 1
printValue(new Valuable(true)) // true
printValue(new Valuable('hello')) // hello
printValue(new Valuable([1, 2, 3])) // [1, 2, 3]

```

### 10.2 제네릭타입 제약
* 제네릭 타입제약 generic type constraint은 타입변수에 적용할 수 있는 타입의 범위를 제한할 떄 사용
  - `<최종타입1 extend 타입1, 최종타입2 extend 타입2>(a: 최종타입, b: 최종타입2, ...) {}`

##### src/1001/printValueT.ts
* 제네릭타입 제약 구문을 사용
* 다음처럼 구현하면 않된다. 왜냐하면 o의 T입장에서 타입 T는 IValuable<IValuable<T>로 해석하기 때문이다
  - 에러 : const printValueT = <T extends IValuable<T>>(o: T) => console.log(o.value)


```ts
import {IValuable} from './IValuable'

// Q는 value 속성의 타입을 나타낸다.
// T는 IValuable<Q>를 확장하는 타입입니다. 즉, T는 value 속성을 가진 객체여야 한다.
export const printValueT = <Q, T extends IValuable<Q>>(o: T) => console.log(o.value)
export {IValuable}
```
##### src/1001/printValueT-test.ts

```ts
import {IValuable, printValueT} from './printValueT'
import {Valuable} from './Valuable'

printValueT(new Valuable(1)) // 1
printValueT( {value: true})  // true
```

#### new 타입제약
* 타입스크립트는 예를 들어 'const create = <T>(type: T):T => new type()'와 같이
* `타입의 타입을 허용하지 않는다.` 따라서, 아래와 같은 구문으로 작성해야 한다
  - `const create = <T extends {new(): T}>(type: T):T -> new type()`
    - new()는 클래스를 생성할 수 있는 타입을 나타낼 때 사용함.
    - new (args): Type 형태로 사용하여 생성자 함수 타입을 정의할 수 있음.
    - 제네릭과 결합하면 더 유연한 인스턴스 생성 로직을 만들 수 있음.
  - {new(): T}는 new()=> T와 같은 의미
    - `const create = <T>(type: new() => T):T => new type()`
  - new()연산자를 type에 적용하면서 type의 생성자 쪽으로 매개변수를 전달해야 할 때 다음처럼
    - new(...args)구문을 사용한다.
    - `const create = <T>(type: {new(...args): T}, ...args):T => new type(...args)`

##### src/1001/create.ts
    
```ts
//
// export const create = <T>(type: { new (...args): T }, ...args): T => new type(...args)
export const create = <T>(type: { new (...args: any[]): T }, ...args: any[]): T => new type(...args)
    
/*
  type: { new (...args: any[]): T }
  이 부분이 핵심. 이는 생성자 함수 타입을 정의하는 TypeScript 문법이다.
  
  'new (...args: any[]): T'의 의미
   * new 키워드가 있는 타입 → 클래스를 생성할 수 있는 타입이라는 의미.
   * (...args: any[]) → 어떤 매개변수든 받을 수 있다.
   * : T → 이 생성자는 T 타입의 객체를 반환한다.
*/
```

### create.ts의 상세한 설명
    
```
1. create 함수의 역할과 new() 사용법 상세 설명
   - create = <T>(type: { new (...args: any[]): T }, ...args: any[]): T => new type(...args);
   - 이 함수는 어떤 클래스든 생성할 수 있는 제네릭 팩토리 함수이다.
   - 이 함수의 역할은 클래스를 동적으로 생성하는 함수를 제공하는 것이다.
   - 즉, create(Car, "Tesla")처럼 호출하면 new Car("Tesla")와 동일한 역할을 합니다.

2. 코드분석
   1) 제네릭 <T>
      - export const create = <T>(...): T =>
      - <T>는 제네릭 타입으로, 이 함수가 생성할 객체의 타입을 나타냅니다.
      - 예를 들어, Car 클래스를 사용하면 T는 Car가 됩니다.
    
   2) type: { new (...args: any[]): T }  
      - type: { new (...args: any[]): T }
      - 이는 생성자 함수 타입을 정의하는 TypeScript 문법입니다.
      - 이 타입의 의미:
        - new (...args: any[]): T
          - new 키워드가 있는 타입 → 클래스를 생성할 수 있는 타입이라는 의미입니다.
          - (...args: any[]) → 어떤 매개변수든 받을 수 있습니다.
          - : T → 이 생성자는 T 타입의 객체를 반환합니다.  

3. 예제코드에서 new (...args: any[]): T가 실제로 어떻게 적용되는지 확인해 보겠습니다.
   - 아래 코드에서 Car와 Person 클래스는 new (...)를 통해 생성될 수 있기 때문에
   - { new (...args: any[]): T } 타입과 일치합니다.
```
```ts
class Car {
  model: string;
  constructor(model: string) {
    this.model = model;
  }
}

class Person {
  name: string;
  constructor(name: string) {
    this.name = name;
  }
}

// `type`의 역할을 확인
const carConstructor: { new (model: string): Car } = Car;
const personConstructor: { new (name: string): Person } = Person;

const myCar = new carConstructor("Tesla"); // new Car("Tesla")와 동일
const myPerson = new personConstructor("Alice"); // new Person("Alice")와 동일
```
```
   3) ...args: any[]
      - ...args는 **가변 매개변수(rest parameter)**를 의미합니다.
      - any[]는 어떤 타입의 매개변수든 받을 수 있음을 의미합니다.
      - 이를 통해, 생성자에 전달할 여러 개의 인수를 받을 수 있습니다.
    
    
   4) new type(...args)    
      - type은 { new (...args: any[]): T } 타입이므로, 클래스(생성자)여야 합니다.
      - 따라서 new type(...args)는 새로운 객체를 생성하는 코드입니다.
      - 예를 들어, Car 클래스라면 new Car(...args)와 같습니다.    

4. create 함수 사용 예제    
```
```ts
class Car {
  model: string;
  constructor(model: string) {
    this.model = model;
  }
}

class Person {
  name: string;
  age: number;
  constructor(name: string, age: number) {
    this.name = name;
    this.age = age;
  }
}

// create 함수를 사용하여 객체 생성
const myCar = create(Car, "Tesla");
const myPerson = create(Person, "Alice", 30);

console.log(myCar.model); // Tesla
console.log(myPerson.name, myPerson.age); // Alice 30
```
```
5. create 함수의 장점
   
   1) 재사용 가능 : 클래스마다 new ClassName(...)을 직접 호출할 필요 없이, 
      동적으로 클래스인스턴스를 생성할 수 있다.
   2) 유연성 : 어떤 클래스든 받을 수 있으므로, 다양한 타입을 처리할 수 있습니다.
   3) 제네릭 타입 안전성 유지 : create<T>를 사용하면 클래스 타입을 잃지 않고 정확한 객체 타입을 
      반환할 수 있습니다.
    
6. 결론
   - { new (...args: any[]): T }는 클래스를 생성할 수 있는 타입을 나타냅니다.
   - new type(...args)는 동적으로 객체를 생성하는 코드입니다.
   - create<T> 함수는 클래스를 동적으로 생성하는 제네릭 팩토리 함수입니다.
   - 이제 create 함수를 사용하면 어떤 클래스든 동적으로 생성할 수 있습니다!  
```
    
#### 인덱스 타입제약
* 가끔 객체의 일정 속성들만 추려서 아래처럼 좀 더 단순한 객체를 만들어야 할 때가 있다.
```ts
const obj = {name: '홍길동', age: 22, city: '서울', country: '대한민국' }
pick(obj, ['name', 'age'])
```

##### src/1002/pick.ts
    
```ts
export const pick = <T, K extends keyof T>(obj: T, keys: K[]) =>
  keys.map(key => ({[key]: obj[key]}) )
      .reduce((result, value) => ({...result, ...value}), {})
```

##### src/1002/pick-test.ts    
    
```ts
import {pick} from './pick'

const obj = {name:"Jane", age: 22, city: "Seoul", country: "Korea"}  
console.log(
  pick(obj, ['name', 'age']) // { name: 'Jane', age: 22 }
  
  // key에 오류가 있을 경우  
  pick(obj, ['nam', 'agge']) // { name: undefined, age: undefined }    
)    
```
* 상기와 같이 오타가 발생하여 key가 없을 경우 오류가 난다.
* 이러한 상황을 방지하기 위해 `keyof형태로 타입제약을 설정`할 수가 있다.
  - `<T, K extends keyof T>`
  - 이러한 오류를 해결하려면 타입 K가 T의 속성이름(key)라는 것을 알려줘야 한다.

### 10.3 대수 데이터 타입
* 객체지향언어에서 `ADT라는 용어는 추상데이터타입 abstract data type을 의미`하지만
* 함수형언어에서는 `대수데이터타입 algebraic data type을 의미`
* 타입스크립트에서 대수데애터타입은 `합집합타입 union type과 교집합타입 intersection type`이 있다.
* 객체지향언어들은 상속(inheritance)에 기반을 두고 타입을 분류하는 경향이 있지만
* 상속에만 의존하면 true, false 단 두 가지 값을 가지는  boolean과 같은 타입을 만들기가 어렵다.
* 이 때문에 `함수형 언어들은 상속에 의존하는 타입보다 대수데이터타입을 선호`한다.

#### 합집합 타입
* 합집합 타입은 `or, |기호로 다양한 타입을 연결해서 만든 타입`을 말한다.
```ts
type NumberOrString = number | string
let ns: NumberOrString = 1
ns = 'hello?'
```
#### 교집합 타입
* 교집합 타입은 `and의 의미인 &기호로 다양한 타입을 연결해서 만든 타입`을 말한다.
* 교집합 타입의 대표적인 예는 두 개의 객체를 통합해서 새로운 객체를 만드는 것이다.

##### src/1003/mergeObjects.ts

```ts
export const  mergeObjects = <T, U>(a: T, b: U) : T & U => ({ ...a, ...b })    
``` 
##### src/1003/mergeObjects-test.ts

```ts
import {mergeObjects} from './mergeObjects'

type INameable = { name: string }
type IAgeable = { age: number }

const nameAndAge: INameable & IAgeable = mergeObjects( {name: "Jack"}, {age: 32})
console.log(nameAndAge) // { name: 'Jack', age: 32 }    
```
 
#### 합집합 타입 구분하기
* 다음과 같은 인터페이스 세 개가 있다고 가정
    
```ts
// 1. 인터페이스
interface ISquare {size: number}
interface IRectangle {width: number, height: number}
interface ICircle {radius: number}

// 2. 각각의 객체
const square: ISquare = {size: 10}
const rectangle {width: 4, height: 5}
const circle {radius: 10}

// 3. 면적계산
// 세 개의 타입을 모두 전달가능한 calcArea()함수가 있다고 가정할 경우
console.log(calcArea(square), calcArea(rectangle), calcArea(circle));

// 4. calcArea()함수는 세 가지타입 ISquare, IRectangle, ICircle을 구분할 수 있어야 한다.
type IShape =  ISquare | IRectangle | ICircle
export const circleArea = (shape: IShape): number => {
  // shape객체가 어떤 타입인지 알 수가 없다.
  return 0;
}
```
* 이전 코드에서  shape객체가 어떤 타입의 객체인지 구분할 수가 없어서 계산코드를 작성할 수가 없다.
* 타입스크립트는 이를 합집합의 타입을 구분할 수 있게 하는 `식별 합집합 discirminated unions 구문을 제공`
    
#### 식별 합집합 구문
* 식별 하지합 구문을 사용하려면 합집합 타입을 구성하는 인터페이스들이 모두 동일명의 속성을 가져야 한다.

##### src/1003/IShape.ts

```ts
export interface ISquare { tag: 'square', size: number }
export interface IRectangle { tag: 'rectangle', width: number, height: number }
export interface ICircle { tag: 'circle', radius: number }

export type IShape = ISquare | IRectangle | ICircle
```

```ts
import {IShape} from './IShape'

export const calcArea = (shape: IShape): number => {
  switch(shape.tag) {
    case 'square': return shape.size * shape.size
    case 'rectangle': return shape.width * shape.height
    case 'circle': return Math.PI * shape.radius * shape.radius
  }
  return 0
}    
```    
##### src/1003/calcArea-test.ts    
    
```tsimport {calcArea} from './calcArea'
import {IRectangle, ICircle, ISquare} from './IShape'

const square: ISquare = {tag: 'square', size: 10 }
const rectangle: IRectangle = {tag: 'rectangle', width: 4, height: 5}
const circle: ICircle = { tag: 'circle', radius: 10 }

console.log( 
  calcArea(square), calcArea(rectangle), calcArea(circle) // 100 20 314.1592653589793
)     
```

### 10.4 타입가드
* 다음과 같은 두 개의 타입있다고 가정

##### src/1004/BirdAndFish.ts

```ts
export class Bird { fly() { console.log(`I'm flying.`) } }
export class Fish { swim() { console.log(`I'm swimming.`) } }    

// 아래와 같이 함수를 구현할 때 매개변수 o가 Bird or Fish이기 때문에 코드작성이 모호해 질 수가 있다.
// 즉, 합집합타입 (Bird | Fish)의 객체가 구체적으로 알아야 한다. 
import {Bird, Fish} from './BirdAndFish'

export const flyOrSwim = (o: Bird | Fish): void => {
  o.fly() // ???
}
```
#### insetancdof 연산자
* 합집합타입의 객체의 인스턴스인지 확인
* 사용법 : `객체 instanceof 타입`

##### src/1004/BirdAndFish.ts

```ts
import {Bird, Fish} from './BirdAndFish'

export const flyOrSwim = (o: Bird | Fish): void => {
  if( o instanceof Bird) {
    (o as Bird).fly() 
  } else if( o instanceof Fish) {
    (o as Fish).swim() // 또는 (<Fish>o).swim()
  }
}    
```

#### 타입가드
* 타입스크립트에서는 instancdof는 자바스크립트와는 다르게 `타입가드 type guard`기능이 있다.
* 이 의미는 타입을 변환하지 않은 코드 때문에 `프로그램의 비정상종료를 막는다`는 것이다.

##### src/1004/flyOrSwim.ts

```ts
import {Bird, Fish} from './BirdAndFish'

export const flyOrSwim = (o: Bird | Fish): void => {
  if( o instanceof Bird) {
    o.fly() 
  } else if( o instanceof Fish) {
    o.swim() 
  }
}      
```
##### src/1004/flyOrSwim-test.ts    
    
```ts
import {Bird, Fish} from './BirdAndFish'
import {flyOrSwim} from './flyOrSwim'

[new Bird, new Fish]
  .forEach(flyOrSwim) // I'm flying. I'm swimming.    
```

#### is 연산자를 활용한 사용자 정의 타입가드 함수 제작
* `함수의 반환타입 부분에 is라는 이름의 연산자를 사용해서 타입가드 기능을 구현`
  - `변수 is 타입`   
  
##### src/1004/isFlyable.ts    

```ts
import {Bird, Fish} from './BirdAndFish'

export const isFlyable = (o: Bird | Fish): o is Bird => {
  return o instanceof Bird
} 
```
##### src/1004/isSwimmable.ts        
    
```ts
import {Bird, Fish} from './BirdAndFish'

export const isSwimmable = (o: Bird | Fish): o is Fish => {
  return o instanceof Fish
}    
```

##### src/1004/swimOrFly.ts   
* 사용자정의 타입가드함수는 if문을 사용해야 한다.

```ts
import {Bird, Fish} from './BirdAndFish'
import {isFlyable} from './isFlyable'
import {isSwimmable} from './isSwimmable'

export const swimOrFly = (o: Fish | Bird) => {
  if(isSwimmable(o))
    o.swim()
  else if(isFlyable(o))
    o.fly()
}
```
##### src/1004/swimOrFly-test.ts    

```ts
import {Bird, Fish} from './BirdAndFish'
import {swimOrFly} from './swimOrFly'

[new Bird, new Fish].forEach(swimOrFly) // I'm flying. I'm swimming.
```

### 10.5 F-바운드 다형성

#### 실습 프로젝트 구성

```bash
npm init --y
npm i -D typescript ts-node @types/node
mkdir -p src/1005/test
mkdir -p src/1005/classes
mkdir -p src/1005/interfaces
tsc --init 
```

##### tsconfig.json

```json
{
  "compilerOptions": {
    "module": "commonjs",
    "esModuleInterop": true,
    "target": "es5",      
    "moduleResolution": "node",      
    "outDir": "dist",
    "baseUrl": ".",
    "sourceMap": true,
    "downlevelIteration": true,
    "noImplicitAny": false,
    "paths": { "*": ["node_modules/*"] }
  },
  "include": ["src/**/**/*"]
}

```
#### this 타입과 F-바운드 다형성(F-bound polymorphism)
* 타입스크립트에서 `this키워드 타입으로도 사용`된다.
* this가 타입으로 사용되면 객체지향언어에서 의미하는 다형성효과가 나는데
* 일반적인 다형성과 구분하기 위해 `this타입으로 인한 다형성을 F-바운드다형성`이라고 한다.

##### 1) F-바운드타입
* F-바운드타입이란? `자신을 구현하거나 상속하는 서브타입을 포함하는 타입`을 말한다. 

##### src/1005/interfaces/IValueProvider.ts
* IValueProvider<T>타입은 특별히 자신을 상속하는 타입이 포함되어 있지 않은 일반타입이다.
    
```ts
export interface IValueProvider<T> {
  value(): T
}
```
    
##### src/1005/interfaces/IAddable.ts
* IAddable<T>는 add메서드가 내가 아닌 `나를 상속하는 타입을 반환하는 F-바운드타입`이다.

```ts
export interface IAddable<T> {
  add(value: T): this
}
```

##### src/1005/interfaces/IMultiplyable.ts

```ts
export interface IMultiplyable<T> {
  multiply(value: T): this
}
```

##### src/1005/interfaces/index.ts
* 코드작성을 편하게 하기 위한 index.ts를 작성
    
```ts
import {IValueProvider} from './IValueProvider'
import {IAddable} from './IAddable'
import {IMultiplyable} from './IMultiplyable'

export {IValueProvider, IAddable, IMultiplyable}
```
    
##### 2)IValueProvider<T> 인터페이스의 구현

##### src/1005/classes/Calculator.ts
* _value를 private로 정의해서 value()메서드로 접근
    
```ts
import {IValueProvider, IAddable, IMultiplyable} from '../interfaces'

export class Calculator implements IValueProvider<number>, IAddable<number>, IMultiplyable<number> {
  constructor(private _value: number = 0) {}
  value(): number { return this._value }
  add(value: number): this { this._value = this._value + value; return this }
  multiply(value: number): this { this._value = this._value * value; return this }
}
```

##### src/1005/classes/StringComposer.ts
* _value를 private로 정의해서 value()메서드로 접근
    
```ts
import {IValueProvider, IAddable, IMultiplyable} from '../interfaces'

export class StringComposer implements IValueProvider<string>, IAddable<string>, IMultiplyable<number> {
  constructor(private _value: string = "") {}
  value(): string { return this._value } 
  add(value: string): this { this._value = this._value.concat(value); return this }
  multiply(repeat: number): this {
    const value =  this.value()
    for(let index=0; index < repeat; index++)
      this.add(value)
    return this
  }
}
                                    
```    
                                    
##### 3) IAddable<T>, IMultiplyable<T> 인터페이스의 구현

##### src/1005/test/Calculator-test.ts
    
```ts
import {Calculator} from '../classes/Calcuclator'

const value = (new Calculator(1))
                .add(2) // 3
                .add(3) // 6
                .multiply(4) // 24
                .value()
console.log(value) // 24
```

##### src/1005/test/StringComposer-test.ts
    
```ts
import {StringComposer} from '../classes/StringComposer'

const value = new StringComposer("hello")
                .add(" ")     // hello
                .add("world") // hello world
                .add("!")     // hello world!
                .multiply(3)  // hello world!hello world!hello world!hello world!
                .value()
console.log(value) // hello world!hello world!hello world!hello world!
```    
* IAddable<T>의 add메서드나 IMultiplyable<T>의 multiply메서드는 자신을 구현한 클래스에 따라
* 반환타입은 Calculator가 되기도 하고 StringComposer가 되기도 한다.
* 즉, this는 어떤 때는 Calculator가 되기도 하고 StringComposer가 되기도 한다.
* 이런 방식으로 동작하는 것을 F-바운드다형성이라고 한다.

### 10.6 nullable타입과 프로그램 안정성

#### 실습 프로젝트 구성

```bash
npm init --y
npm i -D typescript ts-node @types/node
mkdir -p src/1006/test
mkdir -p src/1006/option
tsc --init 
```

```json
{
  "compilerOptions": {
    "module": "commonjs",
    "esModuleInterop": true,
    "target": "es2015",      
    "moduleResolution": "node",      
    "outDir": "dist",
    "baseUrl": ".",
    "sourceMap": true,
    "downlevelIteration": true,
    "noImplicitAny": false,
    "paths": { "*": ["node_modules/*"] }
  },
    "include": ["src/**/**/*"]
}
```
#### nullable타입이란?
* JS와 TS는 변수가 초기화 되지 않으면 undefined라는 값을 기본으로 지정한다.
* 그런데, JS와 TS는 undefined와 사실상 같은 의미인 null이 있다.
* TS는 undefined값의 타입은 undefined이고 null값의 타입은 null이다.
* 이 둘은 `사실상 같은 것이므로 서로 호환`이 된다.

##### src/1006/option/nullable.ts
* undefined와 null타입을 nullable타입이라고 한다.
* 이 nullable타입들은 프로그램이 동작할 때 프로그램을 비정상으로 종료시키는 주요 원인이 된다.
* `함수형 언더들은 이런 비정상 종료를 방지하기 위해 연산자나 클래스를 제공`하기도 한다.

```ts
export type nullable = undefined | null
export const nullable: nullable = undefined
```

#### 옵션 체이닝 연산자
* 변수를 초기화 하지 않을 경우 프로그램의 안정성을 해치므로 이런 것을 방지하기 위해
* `옵션체이닝연산자 optional chaining, 널병합연산자 nullish coalescing operator를 제공`하기도 한다.
* JS는 최근에 `물음표기호와 점기호 ?.연산자를 표준으로 채택`, TS는 3.7.2부터 지원하기 시작
* 옵션체이닝연산자는 세이프내비게이션연산자 safe navigation operator라고도 한다.

##### src/1006/optional-chaining-operator.ts

```ts
export interface IPerson {
  name: string,
  age?: number
}
let person: IPerson
//console.log(person.name) // 런타임 오류 발생
console.log(person?.name) /* 런타임 오류 없이 정상적으로 실행되며, 
                             undefined 값이 반환됩니다. */
```

##### src/1006/safe-navigation.ts

```ts
export type ICoordinates = { longitude: number }
export type ILocation = { country: string, coords?: ICoordinates }
export type IPerson  = { name: string, location?: ILocation }

let person: IPerson = {name: "Jack"}
let longitude = person?.location?.coords?.longitude // safe navigation
console.log(longitude) // undefined
if(person && person.location && person.location.coords) {
  longitude = person.location.coords.longitude
}
```
#### 널 병합 연산자
* JS와 TS는 `물음표2개 ??인 널병합연산자도 채택`
* 옵션체이닝 연산자 부분이 undefined가 되면 널병합연산자가 동작해 undefined대신 0을 리턴하는 예

##### src/1006/nullish-coalescing-operator.ts

```ts
export type ICoordinates = { longitude: number }
export type ILocation = { country: string, coords: ICoordinates }
export type IPerson  = { name: string, location: ILocation }

let person: IPerson
// 널 병합 연산자를 사용하여 기본 값 0을 설정
let longitude = person?.location?.coords?.longitude ?? 0
console.log(longitude) // 0
```

#### nullable타입의 함수형 방식 구현
* 함수형 프로그래밍언어인 하스켈에서는 Maybe라는 타입이 있다.
* 스칼라언어는 Maybe타입의 이름을 Option으로 바꿔서 제공한다.
* 스위프트나 코틀린, 러스트와 같은 언어들도 Option혹은 Optional이라는 이름으로 이 타입을 제공

##### src/1006/option/Option.ts
* 생성자가 private로 선언되었기 때문에 new연산자로 인스턴스를 생성할 수 없다.
* 따라서 Option객체는 `Option.Some(값) 또는 Option.None형태로만 생성`할 수 있다.

```ts
import {Some} from './Some'
import {None} from './None'

export class Option {
  private constructor() {}
  static Some<T>(value : T) { return new Some<T>(value) }
  static None = new None()
}
export {Some, None}
```
* Option.Some정적메서드는 Some이라는 클래스의 인스턴스를 반환하고 
* Option.None 정적 속성은 None이라는 클래스의 인스턴스이다.
* 함수형 언어들은 보통 어떤 정상적인 값을 가지면 Option.Some(1), Option.Some('hello'),
* Option.Some({name: '홍길동'}), Option.Some([1,2,3])처럼 Some타입입에 값을 저장하고
* undefined, null과 같은 비정상적인 값은 None타입으로 처리하는 경향이 있다.
* 이처럼 값을 미리 Some, None으로 분리하면 옵션체이닝연산자나 널병합연산자가 필요 없다.

##### src/1006/option/IValuable.ts
* 왜 Some이나 None으로 값을 구분하는지 알아보자
* Some과 None은 IValuable<T>과 IFunctor<T>라는 인터페이스를 구현하도록 한다.
* 따라서, 해당인터페이스를 구현하는 Some, None은 getOrElse()메서드를 반드시 구현해야 한다.
    
```ts
export interface IValuable<T> {
  getOrElse(defaultValue: T)
}
```
##### src/1006/option/IFunctor.ts
* 함수형 언어에서는 `map이라는 메서가 있는 타입을 펑터 functor`이라고 부른다.
* Some과 None이 IFunctor인터페이스를 구현한다면 반드시 map()함수도 구현해야 한다.    
   
```ts
export interface IFunctor<T> {
  map<U>( fn: (value: T) => U)
}
```

##### (1) src/1006/option/Some.ts 클래스 구현
* Some클래스의 사용자는 getOrElse메서드를 통해 값을 얻고, map메서드를 통해 값을 변경해야 한다.    
   
```ts
import {IValuable} from './IValuable'
import {IFunctor} from './IFunctor'

export class Some<T> implements IValuable<T>, IFunctor<T> {
  constructor(private value: T) {}
  getOrElse(defaultValue: T) {
    return this.value ?? defaultValue
  }
  map<U>(fn: (T) =>  U) {
    return new Some<U>(fn(this.value))
  }
}
```    
    
##### (2) src/1006/option/None.ts 클래스 구현    
       
```ts
import {nullable} from './nullable'
import {IValuable} from './IValuable'
import {IFunctor} from './IFunctor'

export class None implements IValuable<nullable>, IFunctor<nullable> {
  getOrElse<T>(defaultValue: T | nullable) {
    return defaultValue
  }
  map<U>(fn: (T) =>  U) {
    return new None
  }
}
```
    
##### (3) Som과 None 클래스 사용
##### src/1006/test/Option-test.ts
       
```ts
import { Option } from '../option/Option'

let m = Option.Some(1)
let value = m.map(value => value + 1).getOrElse(1)
console.log(value)

let n = Option.None
value = n.map(value => value + 1).getOrElse(0)
console.log(value)
```
      
#### Option타입과 예외처리
* Option타입은 부수효과가 있는 불순함수(Impure)를 순수함수(pure)로 만드는데 효과적이다.
* JS의 parseInt함수는 매개변수가 문자열일 경우 NaN이라는 값을 리턴한다. 

##### src/option/parseNumber.ts
    
```ts
import {Option} from './Option'
import {IValuable} from './IValuable'
import {IFunctor} from './IFunctor'

export const parseNumber = (n: string): IFunctor<number> & IValuable<number> => {
  const value = parseInt(n)
  return isNaN(value) ? Option.None : Option.Some(value)
}  
```    

##### src/test/parseNumber-test.ts
    
```ts
import {parseNumber} from '../option/parseNumber'

let value = parseNumber("1")
              .map(value => value + 1)
              .map(value => value * 2)
              .getOrElse(0)
console.log(value) // 4

value = parseNumber("hello world") 
              .map(value => value + 1)
              .map(value => value * 2)
              .getOrElse(0)
console.log(value) // 0    
```       
    
##### src/option/parseJson.ts
* JSON포맷이 아니라면 예외를 발생시키는 불순함수이지만  try/catch문과 Option을 활용해 순수함수가 되었다.
    
```ts
import {Option} from './Option'
import {IValuable} from './IValuable'
import {IFunctor} from './IFunctor'

export const parseJson = <T>(json: string): IValuable<T> & IFunctor<T> => {
  try {
    const value = JSON.parse(json)
    return Option.Some<T>(value)
  } catch(e) {
    return Option.None
  }
} 
```    
    
##### src/test/parseJson-test.ts
    
```ts
import {parseJson} from '../option/parseJson'

const json = JSON.stringify( {name: "Jack", age: 32}  )
let value = parseJson(json).getOrElse({})
console.log(value) // { name: 'Jack', age: 32 }

value = parseJson("hello world").getOrElse({})
console.log(value) // {}
```